In [ ]:
%pip install pwntools

In [ ]:
import base64
import codecs
from Crypto.Util.number import long_to_bytes

from challenge_files import pwntools_example

In [ ]:
result = pwntools_example.json_recv()
result

We can see that the returned message has two parts:
- The first part is called "type" and it reprensents the encoding type used to encode the message. (see variable ENCODINGS at challenge_files/13377.py).
- The second part is called "encoded" and it is the encoded message.

All we need to do is to reverse the encoding and to retuen the decoded message. We can see at challenge_files/13377.py line 53 that when we will do this decoding successfully enough times, the server will return the flag.

In [ ]:
while 'flag' not in result:
    match result['type']:
        case 'base64':
            decoded = base64.b64decode(result['encoded']).decode() # base64.b64decode() converts base64 string to bytes
        case 'hex':
            decoded = bytes.fromhex(result['encoded']).decode() # bytes.fromhex() converts hex string to bytes
        case 'rot13':
            decoded = codecs.decode(result['encoded'], 'rot_13') # rot13 is a Caesar cipher with a shift of 13
        case 'bigint':
            decoded = long_to_bytes(int(result["encoded"], 16)).decode() # int(x, 16) converts hex string to 10-base int
        case 'utf-8':
            decoded = ''.join(map(chr, result['encoded'])) # map(chr, x) converts list of ints to string
    
    pwntools_example.json_send({"decoded": decoded}) #taken from pwntools_example.py
    result = pwntools_example.json_recv()

In [ ]:
result['flag']